<a href="https://colab.research.google.com/github/jadavbindupriya/Sudoku-Game/blob/main/Sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import time
import threading
from IPython.display import display, clear_output

In [110]:
# Timer and game variables
start_time = time.time()
elapsed_time = 0
timer_running = True
selected_cell = None
hint_count = 3  #  Limit hints to 3

In [111]:
# Function to generate a Sudoku puzzle
def generate_sudoku(difficulty="Medium"):
    full_board = np.array([
        [5, 3, 4, 6, 7, 8, 9, 1, 2], [6, 7, 2, 1, 9, 5, 3, 4, 8], [1, 9, 8, 3, 4, 2, 5, 6, 7],
        [8, 5, 9, 7, 6, 1, 4, 2, 3], [4, 2, 6, 8, 5, 3, 7, 9, 1], [7, 1, 3, 9, 2, 4, 8, 5, 6],
        [9, 6, 1, 5, 3, 7, 2, 8, 4], [2, 8, 7, 4, 1, 9, 6, 3, 5], [3, 4, 5, 2, 8, 6, 1, 7, 9]
    ])
    difficulty_levels = {"Easy": 40, "Medium": 30, "Hard": 20}
    num_clues = difficulty_levels.get(difficulty, 30)

    puzzle = full_board.copy()
    mask = np.random.choice([True, False], puzzle.shape, p=[num_clues / 81, 1 - num_clues / 81])
    puzzle[~mask] = 0
    return puzzle, full_board

In [112]:
# Initialize Sudoku
difficulty = "Medium"
sudoku_puzzle, solution = generate_sudoku(difficulty)
original_puzzle = sudoku_puzzle.copy()
user_puzzle = sudoku_puzzle.copy()

In [113]:
# Function to draw Sudoku board
def draw_sudoku(board, message=""):
    global selected_cell
    output.clear_output(wait=True)
    with output:
        fig, ax = plt.subplots(figsize=(6, 6))

        for i in range(10):
            lw = 2 if i % 3 == 0 else 0.5
            ax.plot([0, 9], [i, i], 'k', linewidth=lw)
            ax.plot([i, i], [0, 9], 'k', linewidth=lw)

        for row in range(9):
            for col in range(9):
                num = board[row, col]
                if num != 0:
                    color = 'black' if original_puzzle[row, col] != 0 else 'blue'
                    ax.text(col + 0.5, 8.5 - row, str(num), fontsize=16, ha='center', va='center', color=color)

                # Display row,col number in small font at the top-left of each cell
                ax.text(col + 0.1, 8.9 - row, f"{row},{col}", fontsize=8, ha='left', va='top', color='gray')

                if selected_cell == (row, col):
                    ax.add_patch(plt.Rectangle((col, 8 - row), 1, 1, color='yellow', alpha=0.3))

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim(0, 9)
        ax.set_ylim(0, 9)
        ax.set_frame_on(False)

        plt.show()

        print(f"Time elapsed: {elapsed_time:.1f} seconds")
        print(f"Hints remaining: {hint_count}")
        if message:
            print(message)

In [114]:
# Timer function
def update_timer():
    global elapsed_time, timer_running
    while timer_running:
        elapsed_time = time.time() - start_time
        time.sleep(1)

In [115]:
# Start timer thread
timer_thread = threading.Thread(target=update_timer)
timer_thread.start()

In [116]:
# Function to select a cell
def select_cell(b):
    global selected_cell
    try:
        row, col = map(int, cell_input.value.split(','))
        if 0 <= row < 9 and 0 <= col < 9:
            selected_cell = (row, col)
            draw_sudoku(user_puzzle, f"Selected Cell: ({row}, {col})")
        else:
            raise ValueError
    except ValueError:
        draw_sudoku(user_puzzle, "Enter row,col between 0-8 (e.g., 3,4)")


In [117]:
# Function to update board with user input
def update_sudoku(b):
    global selected_cell, timer_running
    if selected_cell:
        row, col = selected_cell
        num = num_select.value

        if original_puzzle[row, col] == 0:
            if is_valid(user_puzzle, row, col, num):
                user_puzzle[row, col] = num
                if is_puzzle_solved(user_puzzle):
                    timer_running = False
                    draw_sudoku(user_puzzle, f"Puzzle solved in {elapsed_time:.1f} seconds! 🏆")
                else:
                    draw_sudoku(user_puzzle, "Valid move!")
            else:
                draw_sudoku(user_puzzle, "Invalid move!")
        else:
            draw_sudoku(user_puzzle, "Cannot change pre-filled numbers!")
    else:
        draw_sudoku(user_puzzle, "Select a cell first!")

In [118]:
# Function to provide a hint (limited to 3 hints)
def hint(b):
    global hint_count
    if hint_count > 0:
        empty_cells = list(zip(*np.where(user_puzzle == 0)))
        if empty_cells:
            row, col = empty_cells[0]  # Select first empty cell
            user_puzzle[row, col] = solution[row, col]  # Fill with correct value
            hint_count -= 1  # Reduce hint count
            draw_sudoku(user_puzzle, f"Hint used! ({row},{col}) → {solution[row, col]}")
        else:
            draw_sudoku(user_puzzle, "No empty cells left!")
    else:
        draw_sudoku(user_puzzle, "No hints left!")

In [119]:
# Function to reset the game
def reset_game(b):
    global user_puzzle, original_puzzle, sudoku_puzzle, solution, start_time, elapsed_time, timer_running, hint_count
    sudoku_puzzle, solution = generate_sudoku(difficulty_select.value)
    original_puzzle = sudoku_puzzle.copy()
    user_puzzle = sudoku_puzzle.copy()
    hint_count = 3  # Reset hint count

    start_time = time.time()
    elapsed_time = 0
    timer_running = True

    draw_sudoku(user_puzzle, f"Restarted with {difficulty_select.value} difficulty!")


In [121]:
# Function to check if move is valid
def is_valid(board, row, col, num):
    if num in board[row, :] or num in board[:, col]:
        return False
    start_row, start_col = (row // 3) * 3, (col // 3) * 3
    if num in board[start_row:start_row+3, start_col:start_col+3]:
        return False
    return True

In [122]:
# Function to check if the puzzle is solved
def is_puzzle_solved(board):
    return np.all(board != 0)


In [127]:
# UI Elements
difficulty_select = widgets.Dropdown(options=["Easy", "Medium", "Hard"], value="Medium", description="Difficulty:")
cell_input = widgets.Text(placeholder="Enter row,col (e.g., 3,4)", description="Cell:")
num_select = widgets.Dropdown(options=list(range(1, 10)), description="Number:")
select_button = widgets.Button(description="Select Cell")
submit_button = widgets.Button(description="Submit")
hint_button = widgets.Button(description="Hint")
reset_button = widgets.Button(description="Restart")

output = widgets.Output()


In [134]:
# Connecting buttons to functions
select_button.on_click(select_cell)
submit_button.on_click(update_sudoku)
hint_button.on_click(hint)
reset_button.on_click(reset_game)

In [137]:

# Display UI
display(difficulty_select, cell_input, select_button, num_select, submit_button, hint_button, reset_button, output)

# Draw initial board
draw_sudoku(user_puzzle)


Dropdown(description='Difficulty:', options=('Easy', 'Medium', 'Hard'), value='Easy')

Text(value='', description='Cell:', placeholder='Enter row,col (e.g., 3,4)')

Button(description='Select Cell', style=ButtonStyle())

Dropdown(description='Number:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9), value=1)

Button(description='Submit', style=ButtonStyle())

Button(description='Hint', style=ButtonStyle())

Button(description='Restart', style=ButtonStyle())

Output()